In [1]:
import numpy as np
import pandas as pd

In [36]:
class MLP(object):
    model = None

    @staticmethod
    def f(net):
        return ( 1/ (1+ np.exp(-net)) )

    @staticmethod
    def df_dnet(f_net):
        return ( f_net * (1 - f_net) )

    def __init__(self, input_length=2, hidden_length=[3, 3], output_length=1, activation_function=f , d_activation_function=df_dnet):
        hidden_layer = list()
        previous_length = input_length
        for layer_length in hidden_length:
            hidden_layer.append(np.random.rand(layer_length, previous_length+1) - 0.5)
            previous_length = layer_length

        self.model = {
            'input_length': input_length, 
            'hidden_length': hidden_length, 
            'output_length': output_length, 
            'activation_function': activation_function.__func__, 
            'd_activation_function': d_activation_function.__func__,
            'hidden': hidden_layer,
            'output': (np.random.rand(output_length, previous_length+1) - 0.5),
        }

    def forward(self, x):
        # Recuperando valores do modelo
        hidden = self.model['hidden']
        hidden_length = len(self.model['hidden_length'])
        output = self.model['output']
        f = self.model['activation_function']
        df = self.model['d_activation_function']

        # Camadas Escondidas
        net_h = [None] * hidden_length
        f_net_h = [None] * hidden_length
        df_net_h = [None] * hidden_length
        
        
        # Adicionando 1 para multiplicar o Theta.
        previous_layer = np.pad(x, (0, 1), 'constant', constant_values=(1))
        for i in range(hidden_length):
            net_h[i] = np.sum(np.multiply(hidden[i], previous_layer), axis=1)
            f_net_h[i] = f(net_h[i])
            df_net_h[i] = df(f_net_h[i])
            previous_layer = np.pad(f_net_h[i], (0, 1), 'constant', constant_values=(1))

            
        # Camada de Saída
        net_o = np.sum(np.multiply(output, previous_layer), axis=1)
        f_net_o = f(net_o)
        df_net_o = df(f_net_o)

        # Retornando valores do forward.
        return {
            'net_h': net_h,
            'f_net_h': f_net_h,
            'df_net_h': df_net_h,
            'net_o': net_o,
            'f_net_o': f_net_o,
            'df_net_o': df_net_o,
        }

    def backpropagation(self, X, Y, eta=0.1, threshold=1e-3, alpha=0, max_inter = 200000):
        squaredError = 2*threshold
        hidden_length = len(self.model['hidden_length'])
        counter = 0
        
        dE2_dw_o_t = 0
        dE2_dw_h_t = [0] * hidden_length
        while(squaredError > threshold and counter < max_inter):
            squaredError = 0
            # Pra cada valor do conjunto de dados
            for x, y in zip(X, Y):
                # Calculando saída
                results = self.forward(x)
                #  Calculando o erro
                error = (y - results['f_net_o'])                
                squaredError += np.sum(np.power(error, 2))
                

                # Backwards camada de saída
                h = hidden_length - 1
                delta_o = error * results['df_net_o']
                f_net_h = np.pad(results['f_net_h'][h], (0, 1), 'constant', constant_values=(1))
                dE2_dw_o = np.multiply(np.array([-2*delta_o]).T, np.array([f_net_h]))
                
                # Backwards camada escondida
                delta_h = [None] * hidden_length
                dE2_dw_h = [None] * hidden_length
                delta = delta_o
                w_o_kj = self.model['output'][:,0:self.model['hidden_length'][h]] 

                # For reverso
                for i in reversed(range(1, hidden_length)):
                    # Cálculo das adaptações
                    delta_h[i] = np.array([results['df_net_h'][i]]) * np.dot(delta, w_o_kj)         
                    dE2_dw_h[i] = np.multiply(-2*delta_h[i].T,
                        np.pad(results['f_net_h'][i-1], (0, 1), 'constant', constant_values=(1)))
                    # Controlando as iterações
                    delta = delta_h[i]
                    w_o_kj = self.model['hidden'][i][:, 0:self.model['hidden_length'][i-1]]
                    
                delta_h[0] = np.array([results['df_net_h'][0]]) * np.dot(delta, w_o_kj)               
                dE2_dw_h[0] =  np.multiply(-2*delta_h[0].T,
                    np.pad(x, (0, 1), 'constant', constant_values=(1)))
                               
                
                # Aplicar adaptação na saída
                self.model['output'] = self.model['output'] - eta * dE2_dw_o - alpha*dE2_dw_o_t

                # Aplicar adaptação na escondida
                for i in range(hidden_length):
                    self.model['hidden'][i] = self.model['hidden'][i] - eta * dE2_dw_h[i] - alpha*dE2_dw_h_t[i]

                dE2_dw_o_t = dE2_dw_o
                dE2_dw_h_t = dE2_dw_h
                
            squaredError = squaredError / len(X) 
            counter += 1
            if(counter % 1000 == 0):
                print('error %.6lf - iter. %d' % (squaredError, counter))

    # just using another naming patterns for foward and backpropagation
    def fit(self, X, Y, eta=0.1, threshold=1e-3, alpha=0, max_inter = 200000):
        self.backpropagation(X, Y, eta=0.1, threshold=1e-3, alpha=0, max_inter = max_inter)

    def predict(self, x):
        return self.forward(x)['f_net_o']


In [3]:
class RBFN(object):
    def __init__(self, input_shape, hidden_shape, output_shape):
        self.input_shape = input_shape
        self.hidden_shape = hidden_shape
        self.sigmas = None
        self.centers = None
        self.mlp = MLP(input_length=self.hidden_shape,hidden_length=[3], output_length=output_shape)

    def _kernel_function(self, center, x, sigma):
        return np.exp(-(np.linalg.norm(center-x)**2)/(2*sigma**2) )
    
    
    def _kernel_layer_fit(self, X, centers, threshold=1e-5, sig_err=1e-5, max_inter = 1e5):
        ids = np.random.choice(X.shape[0], centers, replace=False)
        centroids = X[ids,]
        error = 2*threshold
        
        count = 0
        while (error > threshold and count < max_inter):
            count += 1
            if(count % 100 == 0):
                print(count);
            distances = []
            for c in centroids:
                distances.append(
                    np.apply_along_axis(lambda x, u: np.linalg.norm(x-u), 1, X, c)
                    )

            ids = np.argsort(np.array(distances).T)[:,0]
            error = 0
            for i in range(centers):
                old = centroids[i, ]
                centroids[i,] = np.mean(X[ids == i], axis=0)
                error = error + np.sqrt(np.sum(np.power(centroids[i,] - old, 2)))

        self.centers = centroids
        self.sigmas = [np.std(X[ids == i])+sig_err for i in range(centers)]
        self.centroid_to_x = ids


    def fit(self, X, Y, eta=0.1, alpha=0, max_inter=10000):
        self._kernel_layer_fit(X, self.hidden_shape,max_inter=max_inter)
        transformed_X = []
        for i in range(self.hidden_shape):
            transformed_X.append(np.apply_along_axis(self._kernel_function, 1, X, self.centers[i], self.sigmas[i]))

        transformed_X = np.array(transformed_X).T
        self.mlp.backpropagation(transformed_X, Y, eta=eta, alpha=alpha, max_inter = max_inter)

        
    def predict(self, x):
        transformed_x = []
        for i in range(self.hidden_shape):
            transformed_x.append(self._kernel_function(x, self.centers[i], self.sigmas[i]))
        return self.mlp.forward(transformed_x)['f_net_o']

### Preparando dataset

In [4]:
# Funcao p/ normalizar uma coluna de um dataframe
def norm_f(col):
    return (col - col.min())/(col.max() - col.min())

# Funcao p/ normalizar todas as colunas de um dataframe
def normalize_df(df):
    return df.apply(norm_f,axis=0)

# Fixing datset
with open('seeds_dataset.txt') as f:
    string = f.read().replace('\t\t','\t')
    with open('seeds_dataset_fixed.txt','w') as f2:
        f2.write(string)

# Loading dataset into dataframe
df = pd.read_csv('seeds_dataset_fixed.txt',header=None, sep='\t')
# Switching class for dummy variable
df = pd.concat([df.iloc[:,:7], pd.get_dummies(df[7],prefix='c')],axis=1)
# Normalizing dataframe to 0-1 interval
df = normalize_df(df)
# Shuffleing dataframe entries
df = df.sample(frac=1)

# Result
df.head(5)

,0,1,2,3,4,5,6,c_1,c_2,c_3
117,0.808310,0.834711,0.734120,0.757883,0.844619,0.301512,0.820286,0.0,1.0,0.0
31,0.462701,0.522727,0.583485,0.483108,0.528154,0.344160,0.349089,1.0,0.0,0.0
72,0.629839,0.685950,0.618875,0.607545,0.687099,0.490697,0.626292,0.0,1.0,0.0
57,0.408876,0.417355,0.839383,0.273086,0.557377,0.049006,0.280158,1.0,0.0,0.0
80,0.560907,0.605372,0.673321,0.549550,0.596579,0.619810,0.670113,0.0,1.0,0.0


In [5]:
# Splitting dataframe in data and target
data = df.iloc[:,:7]
target = df.iloc[:,7:]

### Treinando e testando MLP

In [38]:
# Treinando MLP
mlp = MLP(input_length = data.shape[1], hidden_length = [3] ,output_length = target.shape[1])
mlp.fit(data.values, target.values, max_inter=10000)

error 0.036638 - iter. 1000
error 0.029041 - iter. 2000
error 0.027164 - iter. 3000
error 0.026103 - iter. 4000
error 0.025003 - iter. 5000
error 0.023503 - iter. 6000
error 0.021806 - iter. 7000
error 0.020781 - iter. 8000
error 0.020271 - iter. 9000
error 0.019983 - iter. 10000


In [7]:
# Treinando RBFN
rbfn = RBFN(data.shape[1],3,target.shape[1])
rbfn.fit(data.values, target.values)

error 0.388509 - iter. 1000
error 0.383385 - iter. 2000
error 0.381357 - iter. 3000
error 0.380144 - iter. 4000
error 0.379172 - iter. 5000
error 0.378314 - iter. 6000
error 0.377553 - iter. 7000
error 0.376884 - iter. 8000
error 0.376291 - iter. 9000
error 0.375761 - iter. 10000


In [34]:
# Funções para facilitar o teste
def evaluate(X, Y, classifier, classification=True):
    if(classification):
        score = 0
        for x, y in zip(X,Y):
            pred = classifier.predict(x)
            pred[pred == pred.max()] = 1 
            if(np.allclose(np.round(pred),y)):
                score += 1
        return score/X.shape[0]
    else:
        return 0

In [39]:
# Testando Rbfn
print(f'RBFN Accuracy: {evaluate(data.values, target.values, rbfn)}')

# Testando MLP
print(f'MLP Accuracy: {evaluate(data.values, target.values, mlp)}')

RBFN Accuracy: 0.6714285714285714
MLP Accuracy: 0.9904761904761905
